# 🛒 Retail y Consumo Masivo: demanda, surtido y pérdidas
KPIs: rotación, OSA, margen, shrink, fill rate, tiempo reposición.

## Casos de uso clave
- Forecast de demanda por tienda-SKU
- Optimización de inventarios y surtido
- Detección de pérdidas (shrinkage) y fraude
- Promociones personalizadas y canibalización

In [10]:
# Esqueleto de tabla de hechos (ventas)
from dataclasses import dataclass
@dataclass
class Venta:
    fecha: str
    tienda_id: int
    sku: str
    unidades: int
    precio: float

def ingreso(v: Venta) -> float:
    return v.unidades * v.precio

Venta('2025-01-01', 10, 'SKU123', 2, 4.5), ingreso(Venta('2025-01-01', 10, 'SKU123', 2, 4.5))

(Venta(fecha='2025-01-01', tienda_id=10, sku='SKU123', unidades=2, precio=4.5),
 9.0)

## Arquitectura sugerida
- Ingesta POS/ERP y e-commerce (batch/stream).
- Modelo estrella: Hechos Ventas, Dimensión Producto/Tienda/Calendario.
- Serving: dashboards OSA, alertas shrink, API recomendaciones.

## 💼 Valor y palancas en Retail
- Menos quiebres de stock (OSA ↑) y mejor fill rate → ventas y satisfacción.
- Surtido óptimo por tienda-SKU → margen y rotación.
- Menos pérdidas (shrink/fraude) con alertas y controles.
- Promos efectivas sin canibalizar categorías clave.


In [8]:
# Práctica: KPIs con ventas y productos (toy)
from pathlib import Path
import pandas as pd

# Intento de rutas relativas robustas
candidatos = [
    Path('../../datasets/raw/ventas.csv'),
    Path('../datasets/raw/ventas.csv'),
    Path('datasets/raw/ventas.csv'),
]
ventas_path = next((p for p in candidatos if p.exists()), candidatos[0])
prod_cands = [
    Path('../../datasets/raw/productos.csv'),
    Path('../datasets/raw/productos.csv'),
    Path('datasets/raw/productos.csv'),
]
productos_path = next((p for p in prod_cands if p.exists()), prod_cands[0])

try:
    ventas = pd.read_csv(ventas_path)
    productos = pd.read_csv(productos_path)
    
    # Adaptación a nombres reales de columnas
    ventas['fecha'] = pd.to_datetime(ventas['fecha_venta'])
    # Usar columna 'total' que ya tiene el cálculo precio*cantidad
    ventas['ingreso'] = ventas['total']
    
    # Ingreso bruto por mes y top SKUs (usar ME en vez de M deprecated)
    mensual = ventas.groupby(pd.Grouper(key='fecha', freq='ME'))['ingreso'].sum().reset_index()
    top_sku = ventas.groupby('producto_id')['ingreso'].sum().nlargest(5)
    
    print('Ingreso mensual (muestra):')
    print(mensual.head())
    print(f'\nTotal ingreso: ${mensual["ingreso"].sum():,.2f}')
    print('\nTop 5 SKUs por ingreso:')
    print(top_sku)
except Exception as e:
    print('No se pudo cargar datasets de ejemplo:', e)
    print('Asegura la ruta relativa a datasets/raw/*.csv')


Ingreso mensual (muestra):
       fecha   ingreso
0 2024-01-31   5939.72
1 2024-02-29  11234.49
2 2024-03-31   1004.89

Total ingreso: $18,179.10

Top 5 SKUs por ingreso:
producto_id
2     3999.98
1     2599.98
13    1599.98
5     1399.96
12    1049.97
Name: ingreso, dtype: float64


## 🏗️ De datos a decisiones
- Pipeline ELT: POS/ERP → limpieza → modelo estrella (Hechos Ventas, Dim Producto/Tienda/Calendario).
- Serving: dashboards OSA, alertas de shrink, microservicio de recomendaciones.
- Métricas confiables: definiciones comunes y control de calidad (tests de datos).


In [9]:
# Visual: Top 5 SKUs por ingreso (si datos disponibles)
import plotly.express as px
try:
    top5 = top_sku.reset_index().rename(columns={'producto_id':'SKU','ingreso':'Ingreso'})
    fig = px.bar(top5, x='SKU', y='Ingreso', title='Top 5 SKUs por ingreso')
    fig.show()
except Exception as e:
    print('Sin datos de ejemplo para graficar:', e)


## ✅ Checklist de estrategia de negocio y datos
- Objetivo retail (ingresos/margen/rotación): …
- KPI (OSA, fill rate, shrink, canibalización): …
- Palancas de datos: demanda, inventarios, promos, surtido.
- Datos y calidad: POS/ERP/e-comm, catalogación, tests de datos.
- Gobierno/Privacidad: dominios, contratos, PII mínima.
- SLOs técnicos: latencia de ventas, costos consulta, disponibilidad.
- ROI y supuestos: …
- Riesgos: estacionalidad, ruptura de catálogos, privacidad.


## 🔗 Puente Estrategia ↔ Ingeniería de Datos
- OKR: OSA ≥ 95% → KPI negocio: OSA → KPI datos: freshness inventario <2h, exactitud stock ≥99% → Capacidades: ingesta POS/ERP + control de calidad → Decisión: reabastecimiento y planograma → Impacto: +ventas, -quiebres.

```
[POS/ERP]→[ELT + validaciones]→[Hechos Ventas]→[OSA/Fill Rate]→[Alertas]→[Reposición]→[Δ ingresos]
```


In [11]:
# contrato_de_datos (retail)
contrato_datos = {
    "objetivo_negocio": "OSA ≥ 95%",
    "kpi_negocio": "On Shelf Availability",
    "kpi_datos": {"freshness_inv_horas": "<2", "exactitud_stock": ">=99%"},
    "dataset": ["ventas", "inventario", "productos"],
    "propietario": "Dominio Retail / Abastecimiento",
    "slo": {"latencia_reporte_min": "<=30"},
    "decision": "Reposición y planograma diario",
    "impacto_economico_usd": 180000,
}
print(contrato_datos)


{'objetivo_negocio': 'OSA ≥ 95%', 'kpi_negocio': 'On Shelf Availability', 'kpi_datos': {'freshness_inv_horas': '<2', 'exactitud_stock': '>=99%'}, 'dataset': ['ventas', 'inventario', 'productos'], 'propietario': 'Dominio Retail / Abastecimiento', 'slo': {'latencia_reporte_min': '<=30'}, 'decision': 'Reposición y planograma diario', 'impacto_economico_usd': 180000}


## 📋 Caso de uso: Pipeline OSA y decisión de reabastecimiento

**Contexto de negocio:** Una cadena retail pierde +$2M/año por quiebres de stock (OSA <92%). El Director Comercial necesita OSA ≥95% para cumplir contratos con proveedores y evitar pérdida de ventas.

**Problema sin ingeniería de datos:**
- Inventarios reportados manualmente con 48h de retraso.
- Inconsistencias entre POS, WMS y ERP (mismo SKU con 3 stocks distintos).
- Decisiones de reposición basadas en "feeling" del jefe de tienda.

**Solución con ingeniería de datos:**
1. Pipeline ELT diario: POS + WMS + ERP → Lakehouse (Bronze/Silver).
2. Validaciones automáticas: stock negativo, SKU sin maestro, ventas sin precio.
3. Modelo de datos: Hechos Ventas/Inventario + Dim Producto/Tienda/Calendario.
4. Exposición: API de OSA en tiempo real + dashboard con alertas por tienda-categoría.
5. Integración con sistema de reposición automática.

**Resultado:**
- OSA: 92% → 96% en 3 meses.
- Tiempo de detección de quiebre: 48h → 2h.
- Ahorro anual estimado: $1.8M (ventas recuperadas + menor costo de expedites).


In [12]:
# Ejercicio: validación de calidad de datos en inventario
import pandas as pd

# Simular inventario con problemas comunes
inventario = pd.DataFrame({
    'tienda_id': [101, 102, 103, 104, 105],
    'sku': ['A123', 'B456', None, 'D012', 'E789'],
    'stock': [50, -10, 100, 0, 200],
    'precio': [10.5, 20.0, 15.0, None, 8.5],
})

# Reglas de calidad (Data Quality)
errores = []
if inventario['sku'].isnull().any():
    errores.append(f"SKUs nulos: {inventario['sku'].isnull().sum()}")
if (inventario['stock'] < 0).any():
    errores.append(f"Stock negativo en {(inventario['stock'] < 0).sum()} registros")
if inventario['precio'].isnull().any():
    errores.append(f"Precios faltantes: {inventario['precio'].isnull().sum()}")

print("Errores detectados:")
for e in errores:
    print(f"  ❌ {e}")

# Impacto en negocio
cobertura = (inventario['sku'].notna() & inventario['precio'].notna()).mean()
print(f"\nCobertura de datos útil: {cobertura:.1%}")
print("⚠️ Decisiones de reposición solo confiables si cobertura ≥ 99%")


Errores detectados:
  ❌ SKUs nulos: 1
  ❌ Stock negativo en 1 registros
  ❌ Precios faltantes: 1

Cobertura de datos útil: 60.0%
⚠️ Decisiones de reposición solo confiables si cobertura ≥ 99%


---

## 🧭 Navegación

**← Anterior:** [🎯 Estrategia de Datos en LATAM: de la visión al ROI](01_estrategia_datos_latam.ipynb)

**Siguiente →:** [💳 Finanzas y Banca: riesgo, fraude, rentabilidad →](03_finanzas_banca.ipynb)

**📚 Índice de Negocio LATAM:**
- [🎯 Estrategia de Datos en LATAM: de la visión al ROI](01_estrategia_datos_latam.ipynb)
- [🛒 Retail y Consumo Masivo: demanda, surtido y pérdidas](02_retail_consumo_masivo.ipynb) ← 🔵 Estás aquí
- [💳 Finanzas y Banca: riesgo, fraude, rentabilidad](03_finanzas_banca.ipynb)
- [🏥 Salud y Farmacéutico: calidad, trazabilidad, acceso](04_salud_farmaceutico.ipynb)
- [⚡ Energía y Recursos Naturales: continuidad, costos y seguridad](05_energia_recursos_naturales.ipynb)
- [📶 Telecomunicaciones: churn, ARPU y calidad de red](06_telecomunicaciones.ipynb)
- [🏭 Industria y Manufactura: OEE, calidad y throughput](07_industria_manufactura.ipynb)
- [🚚 Logística y Transporte: costo por entrega, SLA, ocupación](08_logistica_transporte.ipynb)
- [🌾 Agro y Alimentos: rendimiento, calidad y trazabilidad](09_agro_alimentos.ipynb)
- [🏛️ Sector Público y Gobierno: transparencia, servicio y eficiencia](10_sector_publico_gobierno.ipynb)

**🎓 Otros Niveles:**
- [Nivel Junior](../nivel_junior/README.md)
- [Nivel Mid](../nivel_mid/README.md)
- [Nivel Senior](../nivel_senior/README.md)
- [Nivel GenAI](../nivel_genai/README.md)
- [Negocio LATAM](../negocios_latam/README.md)
